# Introduction

In this project, we would like to see a demonstration of your ability to use Python to explore and analyse financial data-series, then present your insights.  The project will prompt you to use graphs and data-visualisations at specific points, but please feel free to supplement your submissions with any other visualisations that you feel is appropriate and helpful.  It is organised into 3 sequential sections - we encourage you to read through all the prompts once, first, and consider the context of the entire project, before diving into any code.


We intend to focus on (and discuss with you) the following aspects of your project:
- Proficiency with the Python standard library (such as I/O operations), data-science libraries (such as pandas and numpy), and data-visualisation libraries (such as matplotlib and ipydatagrid)
- Project structure, including defining package pre-requisites and basic environment management (our preferred environment manager is [conda](https://docs.conda.io/en/latest/))
- Conciseness and efficiency of solution presented, particularly with respect to the scalability of solution (i.e. considerations for extensions and reuse)
  - We are mostly interested in a high level discussion, stopping well short of questions such as "how quick does the code run" (see below)
- Overall code quality, especially with respect to readability (including quality of documentation and comments, as appropriate)
- General comfort with financial instruments, financial data, and markets


We **do not** intend to focus on the following aspects of your project:
- Novel or interesting algorithms to perform any of the analyses
  - This includes computational optimisations or parallelisations considerations
- Specific details of code-formatting and code-style (other than consistency throughout the project)
  - Please use as many code cells as you would like and organise your project to best fit your style, so long as it generally follows the structure and flow of the prompts
- Macro, fundamental, or technical analysis of any financial instrument beyond any discussions indicated in the prompts


If you are familiar and comfortable with git, we *strongly* recommend that you start a git repository for the project and submit the entire repository.

---

# Setup - Environment

Please ensure that your environment is capable of running this Jupyter workbook as well as any python modules and scripts.  The requirements are:

- python >= 3.8
- jupyterlab
- pandas >= 1.2; optional dependancies we will need are:
  - openpyxl
  - matplotlib
- requests
- [yfinance](https://github.com/ranaroussi/yfinance)


If you have any other requirements for your submission, please list them below.  Additionally, if you created a new environment specifically for this project, please include the `requirements.txt`, `environment.yml`, or other similar requirements/configuration file, and specify the environment manager you used below; please also add relevant any notes or comments that you think may be helpful:

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import requests
import yfinance as yf
from datetime import datetime
import statsmodels.api as sm

---

# 1. Initial Data Preparation

We will use the yfinance package to retrieve price history data for the following stock market indices and stocks:
- SPY
- AGG
- F
- GM
- UBER
- TSLA
- GOOG
- SNAP
- VOW3.DE
- EXSA.DE
- EURUSD=X
- GD=F


Additionally, we would like you to add to the list 4 more tickers, consisting of:
- Any **2** tickers for companies domiciled in Europe, and priced in EUR
- Any **2** tickers for companies domiciled in the US, and priced in USD


The time period for our analysis will be from **Dec 31, 2015 to Dec 31, 2021**

## Prompt 1: Retrieving and caching data

Using the [yfinance documentation](https://github.com/ranaroussi/yfinance) as a guide, please retrieve historical data from Yahoo Finance for the 16 instruments listed above.  Please save the raw data download (without any data cleaning or processing) as a `pickle` in your project directory.  Please use the pickled pandas object as a cache for the rest of the project and re-load the data you require from the pickle, if necessary.

In [2]:
def get_raw_data(ticker_list,start_date,end_date):

    '''
    Function for getting raw historical data from Yahoo Finance
    '''
    
    data = yf.download(ticker_list,start=start_date,end=end_date)
    
    return data

In [3]:
# parameter settings

# -- default ticker list
ticker_list = (['SPY','AGG','F','GM','UBER','TSLA','GOOG','SNAP','VOW3.DE','EXSA.DE','EURUSD=X','GD=F'])
# -- add-on ticker list: 2 EUR stocks & 2 USD stocks
ticker_list_added = (['SIE.DE','MBG.DE','TM','AAPL'])

ticker_list = ticker_list + ticker_list_added

# -- dates
start_date = datetime(2015,12,31)
end_date = datetime(2021,12,31)

In [4]:
# save raw data to pickle file
raw_data = get_raw_data(ticker_list,start_date,end_date)
pd.to_pickle(raw_data,'raw_data_16tickers.pkl')

[*********************100%***********************]  16 of 16 completed


## Prompt 2: Data wrangling

### Data clean-up

Please realign the data index as a daily data-series and please extract just the "Adj Close" data; i.e. the reshaped dataframe should have dates (without time) as its index and tickers (e.g. `SPY`, `GD=F`) as its columns.  In either a markdown box or in comments, please note any assumptions or adjustments you may have made.  Please export this data to a `.csv` file in your project directory.

In [5]:
# reload raw data
raw_data = pd.read_pickle('raw_data_16tickers.pkl')

#### Some exploration of null values

In [6]:
# 
prices = raw_data['Adj Close'].reset_index().set_index('Date')
prices.head(5)

,AAPL,AGG,EURUSD=X,EXSA.DE,F,GD=F,GM,GOOG,MBG.DE,SIE.DE,SNAP,SPY,TM,TSLA,UBER,VOW3.DE
Date,,,,,,,,,,,,,,,,
2015-12-31,24.130863,92.144112,1.093398,NaN,10.446766,312.299988,28.295013,37.944000,NaN,NaN,NaN,180.575638,118.950439,16.000668,NaN,NaN
2016-01-01,NaN,NaN,1.085906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-04,24.151495,92.109970,1.085399,35.750000,10.357795,309.799988,27.712639,37.091999,50.609890,68.110130,NaN,178.051300,117.422958,14.894000,NaN,105.132751
2016-01-05,23.546274,92.152626,1.082755,36.029999,10.172438,307.250000,26.980511,37.129002,50.603016,68.540146,NaN,178.352417,117.113594,14.895333,NaN,100.974022
2016-01-06,23.085482,92.502426,1.075199,35.529999,9.720162,297.750000,26.015438,37.181000,49.317844,68.269394,NaN,176.102646,114.445328,14.602667,NaN,98.894653


In [7]:
prices.isna().sum()

AAPL         56
AGG          56
EURUSD=X      3
EXSA.DE      46
F            56
GD=F        236
GM           56
GOOG         56
MBG.DE       45
SIE.DE       45
SNAP        349
SPY          56
TM           56
TSLA         56
UBER        900
VOW3.DE      45
dtype: int64

<font color='blue'>

Some comments on missing value processing:

</font>

The reason why `UBER` and `SNAP` have so many missing values is that `UBER`'s IPO was in 2019, while `SNAP`'s was in 2018, thus both of them don't have data available until then. Since in all there are only around 1500 rows, excluding 900 rows is not a good idea. Instead, here we use the following method to fill the missing values:

1. Use `ffill` with limit on maximum number ($\leq3$) of consecutive NaNs and fill the scattered missing values;
2. For tickers with too many missing values after step 1, exclude them from the subsets in `dropna` function;
3. Drop rows containing NaNs in other tickers.

As a result, the length of the data set is kept, while scattered NaNs are filled, which is proper for further data analysis.

In [8]:
def clean_up(raw_data, fill_thresh = 3, keep_NaN_thresh = 3):
    
    '''Function for data cleaning.
    
    Arguments
    ---------
    raw_data: DataFrame
              Dataframe containing raw historical data without any data cleaning or processing.
    
    fill_thresh: int
                 Maximum number of consecutive NaNs to fill.
    
    keep_NaN_thresh: int
                     Exclude a ticker from subset for NaN-detecting if the total NaN count exceeds keep_NaN_thresh.
                     
                     
    Returns
    -------
    cleaned_prices: DataFrame
                    Cleaned Adj Close with date format index.
    
    '''
    
    prices = raw_data['Adj Close'].reset_index().set_index('Date')
    prices.index = prices.index.date
    
    # Fill scattered NaNs with maximum consecutive NaNs limit
    cleaned_prices = prices.fillna(method='ffill',limit=fill_thresh)
    
    # Find the ticker subset which will be further used to dropna
    NaN_check = cleaned_prices.isna().sum().to_dict().items()
    exemption_tick = [key for key, value in NaN_check if value > keep_NaN_thresh]
    
    cleaned_prices.dropna(inplace=True,subset=[ticker for ticker in list(cleaned_prices.columns) 
                                               if ticker not in exemption_tick])
    
    return cleaned_prices

In [9]:
cleaned_prices = clean_up(raw_data)

<font color='blue'>
Check NaN count for each ticker after data clean-up:    
</font>

All scattered NaNs are filled, while the consecutive NaNs are kept, which will be specifically processed according to the specific problem later on.

In [10]:
NaN_count = cleaned_prices.isna().sum()
NaN_count

AAPL          0
AGG           0
EURUSD=X      0
EXSA.DE       0
F             0
GD=F        131
GM            0
GOOG          0
MBG.DE        0
SIE.DE        0
SNAP        303
SPY           0
TM            0
TSLA          0
UBER        874
VOW3.DE       0
dtype: int64

In [11]:
# export cleaned_prices
cleaned_prices.to_csv('cleaned_prices_16tickers.csv')

### File manipulation

Next, in a separate `.py` file, please write a function to read the `.csv` file saved in the last section, and generate some descriptive statistics.  Please compute at least: minimum, maximum, and mean Adj Close as well as total return over the entire data period; please feel free to compute and show a few (2 or 3) additional statistics as well if you would like to fill out the table.


Specifically, the function should accept as input:
  ```
  filepath: str and/or Path
    File path to a .csv data with time-series data
  ```


and returns:
  ```
  Pandas dataframe of descriptive statistics; the index should be tickers, matching the columns in the input csv file, and the columns should be appropriate labels for the descriptive statistics you are tabulating 
  ```
  
Please specify the filename of the `.py` file you have written:

<font color='blue'>

des_stats_16tickers.py
    
</font>

Please import the function you have just written, pass the `.csv` file you generated earlier on to it, and display the results.

In [12]:
prices = pd.read_csv('cleaned_prices_16tickers.csv',index_col=0)
prices.head(5)

,AAPL,AGG,EURUSD=X,EXSA.DE,F,GD=F,GM,GOOG,MBG.DE,SIE.DE,SNAP,SPY,TM,TSLA,UBER,VOW3.DE
2016-01-04,24.151495,92.109970,1.085399,35.750000,10.357795,309.799988,27.712639,37.091999,50.609890,68.110130,NaN,178.051300,117.422958,14.894000,NaN,105.132751
2016-01-05,23.546274,92.152626,1.082755,36.029999,10.172438,307.250000,26.980511,37.129002,50.603016,68.540146,NaN,178.352417,117.113594,14.895333,NaN,100.974022
2016-01-06,23.085482,92.502426,1.075199,35.529999,9.720162,297.750000,26.015438,37.181000,49.317844,68.269394,NaN,176.102646,114.445328,14.602667,NaN,98.894653
2016-01-07,22.111172,92.493896,1.077900,34.689999,9.416174,295.049988,24.950527,36.319500,47.421001,66.963402,NaN,171.877640,111.728729,14.376667,NaN,95.650841
2016-01-08,22.228090,92.698608,1.092598,34.160000,9.297547,295.000000,24.567829,35.723499,46.864319,66.501534,NaN,169.991074,109.302155,14.066667,NaN,95.734024


In [13]:
from des_stats_16tickers import des_stats
PATH = 'cleaned_prices_16tickers.csv'
des_table = des_stats(PATH)
des_table

ModuleNotFoundError: No module named 'des_stats_16tickers'

---

# 2. Bollinger Bands Analysis

A Bollinger Band is an analytical tool used to study the prices of financial instruments.  Please refer to this [Investopedia article](https://www.investopedia.com/terms/b/bollingerbands.asp) as well as this [Wikipedia article](https://en.wikipedia.org/wiki/Bollinger_Bands), for example, for some common explanations for the definition and interpretation of this analysis.


In this section, we will adapt the definition of the Bollinger Band slightly to fit the data we have.  Specifically, let
\begin{align}
P_t &= \mbox{Adj Close on trading day }\ t, \\
MA\ (k)_t &= \mbox{Moving Average of }\ P_t \mbox{ over the previous }\ k \mbox{ trading days}, \\
\sigma\ (k)_t &= \mbox{Standard Deviation of }\ P_t \mbox{ over the previous }\ k \mbox{ trading days} \\
\end{align}


The $Bollinger\ (k,N)$ series for a financial instrument consists of 4 data series,
\begin{align}
P_t&, \\
MA\ (k)_t&, \\
UB\ (k,N) &= MA\ (k)_t + N*\sigma\ (k)_t, \\
\mbox{and }\ LB\ (k,N) &= MA\ (k)_t - N*\sigma\ (k)_t \\
\end{align}

## Prompt 3: Calculating Bollinger Bands
Please calculate the $Bollinger\ (20,2)$ series for `SPY`, and display the results graphically.

In [ ]:
%matplotlib notebook  

def bollinger_bands(ticker,cleaned_prices,k,N,plot_flag=True):
    
    price = cleaned_prices[ticker]
    
    MA = price.rolling(k).mean()
    UB = MA + N * price.rolling(k).std()
    LB = MA - N * price.rolling(k).std()
    
    bbands = pd.DataFrame({'Price_'+ ticker: price,
                          'MA_'+ ticker: MA,
                          'UB_'+ ticker: UB,
                          'LB_'+ ticker: LB}).dropna()
    
    # For plotting
    if plot_flag:
        ax = bbands.plot()
        ax.set_title(f'Bollinger Bands For {ticker}, k={k},N={N}',fontsize=18)
        ax.set_xlabel('Date',fontsize=12)
        ax.set_ylabel('Price',fontsize=12)
    
    return bbands

In [ ]:
_ = bollinger_bands('SPY',cleaned_prices,20,2)

Please calculate 2 more data series, the $Bollinger\ (60,1.5)$ series for the `SPY`, and the $Bollinger\ (k,N)$ series for the `SPY` for a different pair of any $(k,N)$ of your choice. Please visualise your findings, and referring to those graphs, provide some brief written comments on the following questions:
- Please describe the relationship between the parameters $(k,N)$ and the analytical results
- What statistical attributes for any given price time-series would potentially make the Bollinger Bands less and more useful as the basis for a trading strategy?


**Please be prepared to discuss these results and comments further**.

<font color='blue'>
Relationship between (k,N) and the analytical results:
</font>

Higher k leads to smoother MA/UB/LB curve; higher N leads to broader yet more volatile UB/LB, which makes sense since N controls how many standard deviations are included between UB and LB.

<br>

<font color='blue'>
Statistical attributes:
</font>

1. The proportion of prices above UB or below LB should be similar to the "tail" probability of a distribution. E.g. for randomly distributed X, the probability of X's value lying between $\pm2\sigma$ is about 95.6%. Thus, a Bollinger Band is more reasonable if 95.6% of the prices lie within UB and LB.

<div>
<img src="https://cdn.scribbr.com/wp-content/uploads/2020/10/normal-distribution.png", width=320, heigth=240>
</div>

2. The std-to-mean ratio should be relatively large, so that the buy/sell signals can be more significant and the constructed strategy can be both more reliable, more executable and more likely to make a profit.


In [ ]:
_ = bollinger_bands('SPY',cleaned_prices,60,1.5)

In [ ]:
_ = bollinger_bands('SPY',cleaned_prices,10,3)

## Prompt 4: Extending the analysis
One way to think about how useful a particular $Bollinger\ (k,N)$ series may be as a trading signal is to consider how often the price series stays within $UB\ (k,N)$ and $LB\ (k,N)$.  Please propose a calculation and simple algorithm to capture this statistic, which we can denote as $W\ (k,N) = \mbox{Percentage of of the data series }P_t,\ such\ that\ LB\ (k,N) < P_t < UB\ (k,N)$.  Then, please choose **any 4 distinct pairs of $(k,N)$ as well as the pair $(20,2)$** (i.e. 5 pairs), and compute the corresponding $W\ (k,N)$ for `SPY` and present them in a table.

In [ ]:
def cal_pct_within(ticker,cleaned_prices,k,N,plot_flag):
    
    bbands = bollinger_bands(ticker,cleaned_prices,k,N,plot_flag)
    W = (sum((bbands['LB_'+ticker]<bbands['Price_'+ticker]) & (bbands['Price_'+ticker]<bbands['UB_'+ticker]))
         /len(bbands))
    
    return W

In [ ]:
k_list = [10,20,30,60,60]
N_list = [3,2,1.5,1.5,3]

W_table = pd.DataFrame({'k':k_list,'N':N_list})
for i,tick in enumerate(cleaned_prices.columns.values): # cleaned_prices.columns.values = tick_list
    # print(i,tick)
    W_list = []
    for i in range(len(k_list)):
        W_list.append(cal_pct_within(tick,cleaned_prices,k_list[i],N_list[i],False))
    W_table['W_'+tick] = W_list

In [ ]:
W_table[['k','N','W_SPY']]

Cross tabulate $W\ (k,N)$ for those same 5 pairs of $(k,N)$ for all 16 tickers we have used for this exercise, and provide some brief written comments to address the following questions:
- Qualitatively describe how you would design an algorithm to arrive at the optimal $W\ (k,N)$
- Qualitatively propose other metrics that you may use to evaluate how useful any particular $Bollinger\ (k,N)$ series is


**Please be prepared to discuss these results and comments further**.

In [ ]:
W_table

<font color='blue'>
Arrive at the optimal W(k,N):
</font>

Similar to 'What would potentially make the Bollinger Bands less and more useful', the optimal (k, N) should satisfy both 
1. W should be similar to 'non-outlier' proportion of a specific distribution; 
2. std-to-mean should be relatively large.

Thus, the algorithm should be: 
1. Determine the target non-outlier proportion & std-to-mean;
2. For different (k,N) combinations, find the (k,N) which has the closest non-outlier proportion & std-to-mean compared to the target;
3. If necessary, backtest and calculate the PNL and try on different thresholds in step 1.

<br>
<font color='blue'>
Other metrics that can be used in market timing:
</font>

1. The correlation between $W_{ML}$(the proportion of prices lying between MA & LB) and $R_{t+\Delta t}$(return of following period with a specific window length);
2. The correlation between $W_{UM}$(the proportion of prices lying between UB & MA) and $R_{t+\Delta t}$(return of following period with a specific window length).

_[**Note - this is entirely optional:** If there are any other insights that you have gleaned in this section about Bollinger Bands or the tickers we have used for this exercise, please feel free to add to this section below and present your findings.]_

<font color='blue'>
The time when the relationship between $W_{ML}$ and $W_{UM}$ change from $W_{ML}<W_{UM}$ to $W_{ML}>W_{UM}$ might be a signal for trading strategy.
</font>

---

# 3. Statistical attributes of financial data series

Often, it can be useful to transform price series into return series (and in fact, the vast majority of academic literature addressing financial instruments is written based on returns series analysis). In this section, we will transform the data we have and perform some simple statistical exercises and take some first steps towards thinking about portfolio construction.

## Prompt 5: Computing daily returns series



We want to make sure that we are measuring returns for all the tickers on the same currency basis.  Specifically, `VOW3.DE`, `EXSA.DE`, and the two European tickers that you have chosen have price series denominated in `EUR$` Fortunately for us, we have the `EURUSD=F` ticker, which gives us the `EUR`-to-`USD` exchange rate each day (i.e. the Adj Close for `EURUSD=F` tells you how many `USD$` `EUR$1.00` will buy that day).  Please compute a new dataframe, `Adj USD Close`, which is the `Adj Close` dataframe denominated in `USD$`.

In [ ]:
cleaned_USD_prices = cleaned_prices.copy()
for col in cleaned_prices.columns:
    # select EUR tickers
    if '.DE' in col:
        cleaned_USD_prices[col] = cleaned_prices[col]*cleaned_prices['EURUSD=X']

In [ ]:
cleaned_USD_prices.head()

Please generate a dataframe, `Ret`, for the daily returns for all 16 tickers in our new `Adj USD Close` dataset (where, to be specific, $Ret_t=\frac{P_t}{P_{t-1}}-1$), and please tabulate the following descriptive statistics for each ticker:
- Annualised return
- Annualised standard deviation of daily returns

Please feel free to add a few more (no more than 3) statistics that you think may be interesting.

In [ ]:
Ret = cleaned_USD_prices.pct_change()
Ret.head()

In [ ]:
from empyrical.stats import annual_return,annual_volatility,max_drawdown,sharpe_ratio,sortino_ratio

def Annualised_return(ret):
    
    ret_ = ret.loc[ret.index>=ret.isna().idxmin()]     
    num_years = (len(ret_)+ 1)/252
    cumreturn = (ret_+1).cumprod()[-1]
    Annualised_return = (cumreturn)**(1/num_years) - 1
    
    return Annualised_return 

def Annualised_standard_deviation(ret):
    Annualised_std = np.nanstd(ret)*np.sqrt(252)
    return Annualised_std


SIMPLE_STAT_FUNCS = [Annualised_return,Annualised_standard_deviation,max_drawdown,sharpe_ratio,sortino_ratio]
STAT_FUNC_NAMES = ['Annualised return','Annualised std','Max drawdown','Sharpe ratio','Sortino ratio'] 

def perf_stats_(ret):
    stats = pd.Series(dtype='float64')
    for stat_name,stat_func in zip(STAT_FUNC_NAMES,SIMPLE_STAT_FUNCS):
        stats[stat_name] = stat_func(ret.dropna())
    return stats

RET_statistics = pd.DataFrame()
for i,tick in enumerate(cleaned_prices.columns.values): 
    #print(i,tick)
    df1 = perf_stats_(Ret[tick]).to_frame(name=tick)
    RET_statistics = pd.concat([RET_statistics,df1],axis=1)
RET_statistics = RET_statistics.T
RET_statistics

## Prompt 6: Describing relationships between returns series


One useful way to think about the relationship between financial instruments is to analyse the correlation between their daily returns series.  Please compute the correlation matrix for the full data-series in the `Ret` dataframe and tabulate it.

In [ ]:
Ret.corr()

If you could have only picked 4 ticker out of the 16 to hold in an investment portfolio, in equal weight, over the period of the data, which 4 tickers would likely result in a portfolio with lowest volatility (i.e. lower standard deviation of daily returns)?  _Although we intend to discuss this question mostly from a qualitative perspective, please feel free to substantiate your comments with further quantitative analysis, including graphs and tables, if you feel it would be helpful._

<font color='blue'>
Qualitative intuition: 
</font>
Tickers with negative correlation are more likely to form a portfolio with lowest volatility.

<font color='blue'>
<br>
    
Quantitative analysis:
</font>

1. Get all the combinations of 4 tickers out of 16.
2. For each combination, calculate the equally weighted portfolio return, and then calculate the std of the portfolio.
3. Select the combination with lowest portfolio return std.


In [ ]:
from itertools import combinations
combs = combinations(Ret.columns,4)
min_std = float('inf')
min_comb = []
for comb in list(combs):
    portfolio_std = Ret[list(comb)].mean(axis=1).std()
    if min_std > portfolio_std:
        min_std = portfolio_std
        min_comb = list(comb)

In [ ]:
min_comb, min_std

<font color='blue'>
From the correlation matrix below, we can see the qualitative intuition corresponds with the quantitative analysis result.
</font>

In [ ]:
Ret[min_comb].corr()

We can sometimes study the correlations characteristics of a particular ticker, say for instance for a new company we are interesting in analysing, to make some 
simplifying high-level assumptions.  Please graph the rolling 6-month correlations of `TSLA` to `F`, `GM`, `GOOG`, and `SNAP`, and provide brief written comments to address the following question:
- Given that `F` and `GM` are car manufacturing companies while `GOOG` and `SNAP` are technology services companies, what industry would you think `TSLA` falls into (it does not have to be either of the two industries spanned by the other 4 tickers), and why?


**Please be prepared to discuss these results and comments further**.

In [ ]:
corr_TSLA = Ret[['TSLA','F','GM','GOOG','SNAP']].rolling(120).corr().dropna().swaplevel().loc['TSLA']
corr_TSLA

In [ ]:
corr_TSLA.drop(columns=['TSLA']).plot(title='Rolling 6-month Correlation')

<font color='blue'>
Method for classifying:
</font>

1. For each rolling window, find the stock which has the maximum correlation with `TSLA`.
2. Calculate the proportion of each ticker appearing as the maximum_correlation_ticker.
3. `TSLA` should be more similar to the industry whose stocks have a larger correlation with `TSLA` for most of the time.

In [ ]:
corr_TSLA = corr_TSLA.drop(columns=['TSLA'])
corr_TSLA['MaxCorrTicker'] = corr_TSLA.idxmax(1)
corr_TSLA

In [ ]:
corr_TSLA['MaxCorrTicker'].value_counts(),corr_TSLA['MaxCorrTicker'].value_counts()/len(corr_TSLA['MaxCorrTicker'])

<font color='blue'>
    
From above we can see that in nearly 75% of all 6-month rolling window, either `GOOG` or `SNAP` has the highest correlation with `TSLA`. In addition, there's still 25% of all time where `GM` or `F` has the highest correlation with `TSLA`, and the highest correlation is always significant and positive. Thus, we can conclude that `TSLA` behaves more like a technology services company, but still has the characteristics of car manufacturing companies. 
    
</font>

----
We can also use construct linear regression models to analyse the performance characteristics of returns series.  For example, given that the `SPY` represents (very approximately) the performance of market for Large Cap US companies, if we think that the performance of `UBER` can be described as some function of the return of the broader markets, an idiosyncratic component, plus some noise, we can construct a linear relationship between `SPY` and `UBER` specified by

$$Ret_{UBER} = \alpha + \beta * Ret_{SPY} + \epsilon$$

Using whatever method you feel most comfortable with, please perform a linear regression analysis on this hypothesized relationship using the full period of available data, tabulating your full results below.  **While we do not have specific prompts for written comments, please be prepared to briefly discuss the results of the linear regression analysis, including your methods**.

In [ ]:
from statsmodels.stats.diagnostic import het_white

In [ ]:
# Linear regression model fitting
reg_subset = Ret[['UBER','SPY']].dropna()
Y = reg_subset['UBER']
X = reg_subset['SPY']
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()

In [ ]:
# Test for heteroskedasticity
white_test = het_white(model.resid,model.model.exog)

labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
print(dict(zip(labels, white_test)))

<font color='blue'>
    
From the stats above, there's no significant heteroskedasticity in the residuals.

</font>

In [ ]:
# Test for residual autocorrelation
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(model.resid);

<font color='blue'>
From ACF plot, there are autocorrelation when lag = 4,10 and 12. Thus, an AR model or seasonal model might be a better fit for this regression.             
</font>

In [ ]:
model.summary()

<font color='blue'>
Check for the beta from another calculation method:
</font>

In [ ]:
corr_UBER_SPY = Ret[['UBER','SPY']].dropna().corr().iloc[0,1]
beta_theoretical = corr_UBER_SPY*Ret[['UBER','SPY']].dropna()['UBER'].std()/Ret[['UBER','SPY']].dropna()['SPY'].std()
round(beta_theoretical,4)

In [ ]:
sm.graphics.plot_fit(model,1, vlines=False);